In [60]:
"""
https://leetcode.com/problems/design-a-text-editor/

Design a text editor with a cursor that can do the following:

Add text to where the cursor is.
Delete text from where the cursor is (simulating the backspace key).
Move the cursor either left or right.
When deleting text, only characters to the left of the cursor will be deleted. 
The cursor will also remain within the actual text and cannot be moved beyond it. 
More formally, we have that 0 <= cursor.position <= currentText.length always holds.

Implement the TextEditor class:
TextEditor() Initializes the object with empty text.
void addText(string text) Appends text to where the cursor is. The cursor ends to the right of text.
int deleteText(int k) Deletes k characters to the left of the cursor. 
Returns the number of characters actually deleted.
string cursorLeft(int k) Moves the cursor to the left k times. 
Returns the last min(10, len) characters to the left of the cursor, where len is the number of characters to the left of the cursor.
string cursorRight(int k) Moves the cursor to the right k times. 
Returns the last min(10, len) characters to the left of the cursor, where len is the number of characters to the left of the cursor.
 

Constraints:
1 <= text.length, k <= 40
text consists of lowercase English letters.
At most 2 * 10^4 calls in total will be made to addText, deleteText, cursorLeft and cursorRight.
 

Follow-up: Could you find a solution with time complexity of O(k) per call?

deque()

               cursorP
                |
                V
H - a - b - c - | - d - e - f - g - E

H, E: sentinel nodes
cursorP

"""


# class TextEditor:
#     def __init__(self):
#         self.pos = 0
#         self.str = "|"
#         # self.str[0:self.pos]
#         # if self.pos < len(self.str): self.str[self.pos:]

#     def addText(self, text: str) -> None:
#         # print(f"addText({text})", self.str, self.pos)
#         self.str = self.str[:self.pos] + text + self.str[self.pos:] if self.pos < len(self.str) else ""
#         self.pos += len(text)
#         # print(f"addText({text})", self.str, self.pos)

#     def deleteText(self, k: int) -> int:
#         # abc|def
#         # a|def
#         oldLen = len(self.str)
#         # print(f"deleteText({k})", self.str, self.pos)
#         s = self.str[0:max(0, self.pos - k)]
#         preLen = len(s)
#         s += self.str[self.pos:]
#         self.str = s
#         self.pos = preLen
#         # print(f"deleteText({k})", self.str, self.pos)
#         return oldLen - len(self.str)

#     def cursorLeft(self, k: int) -> str:
#         # abc|def
#         # a bc def
#         # a|bcdef
#         # print(f"cursorLeft({k})", self.str, self.pos)
#         nextPos = max(0, self.pos - k)
#         s = self.str[0:nextPos]
#         s += '|'
#         s += self.str[nextPos:self.pos]
#         s += self.str[self.pos+1:]
#         self.str = s
#         self.pos = nextPos
#         # print(f"cursorLeft({k})", self.str, self.pos)
#         return self.str[max(0,self.pos-10):self.pos]


#     def cursorRight(self, k: int) -> str:
#         # abc|def
#         # abc de f
#         # abc de | f
#         # pos: 3 -> 5

#         # |practice
#         # "" practi | ce
#         # 0->6
#         # print(f"cursorRight({k})", self.str, self.pos)
#         nextPos = min(len(self.str)-1, self.pos + k)
#         s = ""
#         if self.pos > 0:
#             s += self.str[0:self.pos]
#         s += self.str[self.pos+1:nextPos+1]
#         s += '|'
#         s += self.str[nextPos+1:]
#         self.str = s
#         self.pos = nextPos
#         # print(f"cursorRight({k})", self.str, self.pos)
#         return self.str[max(0, self.pos-10):self.pos]


class Node:
    def __init__(self, val, prev=None, next=None):
        self.val = val
        self.prev = prev
        self.next = next


class TextEditor:
    def __init__(self):
        # HEAD - | - TAIL
        self.head = Node("HEAD")
        self.pos = Node('|')
        self.tail = Node("TAIL")
        self.head.next = self.pos
        self.pos.prev = self.head
        self.pos.next = self.tail
        self.tail.prev = self.pos
        # self.print()
    
    def print(self):
        retval = "HEAD"
        p = self.head.next
        while p != self.tail:
            retval += ' ' + p.val
            p = p.next
        retval += ' ' + self.tail.val
        print(retval)

    
    def addText(self, text: str) -> None:
        # A | B
        # A AN | B
        A = self.pos.prev        
        for c in text:
            A.next = Node(c)
            A.next.prev = A
            A = A.next
        A.next = self.pos
        self.pos.prev = A
        # self.print()


    def deleteText(self, k: int) -> int:
        numDel = 0
        #  H - A - B - |
        for i in range(k):
            if self.pos.prev != self.head:
                B = self.pos.prev
                A = B.prev
                A.next = self.pos
                self.pos.prev = A
                numDel += 1
        # self.print()
        return numDel


    def _get10Prev(self):
        retVal = ""
        n = self.pos.prev
        for i in range(10):
            if n != self.head:
                retVal = n.val + retVal
                n = n.prev
        return retVal

    
    def cursorLeft(self, k: int) -> str:
        for i in range(k):
            if self.pos.prev != self.head:
                # A B | C
                # A | B C
                B = self.pos.prev
                A = B.prev
                C = self.pos.next

                A.next = self.pos
                self.pos.prev = A
                self.pos.next = B
                B.prev = self.pos
                B.next = C
                C.prev = B

        # self.print()
        return self._get10Prev()


    def cursorRight(self, k: int) -> str:
        for i in range(k):
            if self.pos.next != self.tail:
                # A | B C
                # A B | C
                A = self.pos.prev
                B = self.pos.next
                C = B.next
                A.next = B
                B.prev = A
                B.next = self.pos
                self.pos.prev = B
                self.pos.next = C
                C.prev = self.pos

        # self.print()
        return self._get10Prev()



#Input
#["TextEditor", "addText", "deleteText", "addText", "cursorRight", "cursorLeft", "deleteText", "cursorLeft", "cursorRight"]
#[[], ["leetcode"], [4], ["practice"], [3], [8], [10], [2], [6]]
#Output
#[null, null, 4, null, "etpractice", "leet", 4, "", "practi"]

textEditor = TextEditor()      #; // The current text is "|". (The '|' character represents the cursor)
textEditor.addText("leetcode") #; // The current text is "leetcode|".
assert(textEditor.deleteText(4) == 4)       #; // return 4
                               # // The current text is "leet|". 
                               # // 4 characters were deleted.
textEditor.addText("practice") #; // The current text is "leetpractice|". 
assert(textEditor.cursorRight(3) == "etpractice")     #; // return "etpractice"
                               #// The current text is "leetpractice|". 
                               #// The cursor cannot be moved beyond the actual text and thus did not move.
                               #// "etpractice" is the last 10 characters to the left of the cursor.
assert(textEditor.cursorLeft(8) == "leet")       #; // return "leet"
                               #// The current text is "leet|practice".
                               #// "leet" is the last min(10, 4) = 4 characters to the left of the cursor.
assert(textEditor.deleteText(10) == 4)      #; // return 4
                               #// The current text is "|practice".
                               #// Only 4 characters were deleted.
assert(textEditor.cursorLeft(2) == "")       #; // return ""
                               #// The current text is "|practice".
                               #// The cursor cannot be moved beyond the actual text and thus did not move. 
                               #// "" is the last min(10, 0) = 0 characters to the left of the cursor.
assert(textEditor.cursorRight(6) == "practi")      #; // return "practi"
                               #// The current text is "practi|ce".
                               #// "practi" is the last min(10, 6) = 6 characters to the left of the cursor.
